In [2]:
#check for nulls
df.isnull().sum()

#get unique values of a dataset
bikedf_dummy['birth year'].unique()


#replace wrong data
#option1
df['Age'] = [np.nan if age=='?' else float(age) for age in df['Age'].values]
#option2
bikedf_dummy['birth year'].replace(to_replace='\\N',value = 0,inplace=True)


#transform a column type from object to int
bikedf_dummy['birth year'] = bikedf_dummy['birth year'].astype(int)



#select df subset ( filter)
Y = bikedf.loc[bikedf["gender"] == 2, "tripduration"]


# 1 - Rename "count" to "riders"
# 2 - Drop "casual" and "registered"
# 3 - Drop only rows where weather == 4
# 4 - Cast "datetime" to actually be a datetime
bike.rename(columns={"count": "riders"}, inplace=True) # solution 1
bike.drop(columns=["casual", "registered"], inplace=True) # solution 2
bike = bike[bike["weather"] != 4] # solution 3
bike["datetime"] = pd.to_datetime(bike["datetime"]) # solution 4


#replace missing values THE MANUAL WAY
#replace missing data using mean
mean_age = income_missing['age'].mean()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(mean_age)
#then check the variance between before and after !
#replace missing data using meadian
median_age = income_missing['age'].median()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(median)
#replace missing data using mode ( most frequent )
mode_age = income_missing['age'].mode()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(mode)


#replace missing values THE SKLEARN WAY
from sklearn.impute import SimpleImputer


#Charts

#heatmap
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm');


#heatmap>|0.5| only
pearson_corr_features = df.corr()

corr_data = np.tril(pearson_corr_features,k=-1) # only looking at bottom heatmap triangle
corr_data = abs(corr_data)>0.50 # only visualizing notable correlations 

# plotting
fig = px.imshow(corr_data, x=pearson_corr_features.columns, y=pearson_corr_features.index,
                color_continuous_scale=px.colors.qualitative.Plotly, width=800, height=800,
                title="Heat map: Feature correlations with >0.5 correlation coefficient")
fig.update(layout_coloraxis_showscale=False)
fig.show()

NameError: name 'df' is not defined

Columns operations

In [ ]:
df.columns.values[1] = 'Courses_Fee'
print(df.columns)

# Rename column name by index
df.rename(columns={df.columns[2]: 'Courses_Duration'},inplace=True)
print(df.columns)

### 1. SUPERVISED MACHINE LEARNING

#### 1. Regression

#### Features engineering

In [3]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)
X_poly = poly.fit_transform(X)

# X_poly is a list-of-lists with each 9 transformed feature values
# printing row 1 as sample preview to compare vs X pre-transformation
print(f'pre-X: {X.iloc[1].values}')
print(f'post-X: {X_poly[[1]]}') 




ss = StandardScaler() # instantiate
X_scaled = ss.fit_transform(X) # fit and transform (scale) X
X_scaled[0, :] # preview post-scaling output from 0th row and all cols

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn import metrics 
lr = LinearRegression()
model = LinearRegression()

X = bike[["season", "temp"]] # get a subset_df with all col+values from season, temp cols
X = pd.get_dummies(data=X, columns=["season"], drop_first=True) # dropping season 1
y = bike["riders"]

model.fit(X, y)


dict(zip(X.columns, model.coef_))

# Make predictions using sklearn's predict() method
y_pred = lr.predict(X)

#### adding cross validation of dataset

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

lr = LinearRegression()
lr.fit(X_train, y_train)

lr.score(X_train, y_train)
lr.score(X_test, y_test)

cross_val_score(lr, X_train, y_train, cv=5)# defaults to linear regression's default scoring, R-squared
cross_val_score(lr, X_train, y_train, cv=5).mean()

#### Addting regularization

In [ ]:
#Ridge
from sklearn.linear_model import RidgeCV
r_alphas = np.logspace(0, 5, 100)

# Cross-validate over our list of ridge alphas.
# alphas: pass an Array of alpha values to try. It is still the Regularization strength
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5).fit(Z_train, y_train)# fitting 5-fold CV on 100 alphas, that is 500 alphas!
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))
ridge_cv.alpha_

In [ ]:
# Imports similar to Ridge, this time for Lasso instead
from sklearn.linear_model import Lasso, LassoCV
# then converts them to alphas between 10^-3 and 1 (that is, in logscale).
l_alphas = np.logspace(-3, 0, 100)

# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=50000).fit(Z_train, y_train);
lasso_cv.alpha_

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_sc, y_train)

# See the gap in training vs testing metric! Overfit!
print(logreg.score(X_train_sc, y_train))
print(logreg.score(X_test_sc, y_test))


# we arrive at the same outcome by setting penalty='none'
logreg_pen_none = LogisticRegression(penalty='none')
logreg_pen_none.fit(X_train_sc, y_train)

# See the gap in training vs testing metric! Overfit!
print(logreg_pen_none.score(X_train_sc, y_train))
print(logreg_pen_none.score(X_test_sc, y_test))

##---------------------------
logreg_cv = LogisticRegressionCV(cv=5, penalty="l1", solver="liblinear")
logreg_cv.fit(X_train_sc, y_train)

# we don't see overfitting now! training-testing metrics are close
print(logreg_cv.score(X_train_sc, y_train))
print(logreg_cv.score(X_test_sc, y_test))

### Side note on stats model

In [ ]:
import statsmodels.api as sm

X = elec[["temp", "workday"]]
X = sm.add_constant(X) # remember that unlike scikit-learn, statsmodels needs this to be explicitly specified
y = elec["demand"]
ols = sm.OLS(y, X).fit()

#### Metrics evaluation

In [ ]:
#metrics
from math import sqrt # alternative to (variable)**0.5 ( imported from sklearn too)

resids = y - y_pred
mse = np.mean(resids**2)
y_bar = np.mean(y)
null_mse = np.mean((y - y_bar)**2)
r_squared = mse/null_mse


metrics.mean_squared_error(y, predictions)
np.sqrt(metrics.mean_squared_error(y, predictions))
metrics.mean_absolute_error(y, predictions)
metrics.r2_score(y, predictions)

#### Charting

In [ ]:
# Charting for linear regression
sns.regplot(x='temp', y='demand', data=elec, ci=None, 
            scatter_kws = {'s': 1}, 
            line_kws = {'color': 'orange'})



In [1]:
sns.lmplot(x = 'body', y = 'brain', data = mammals,
           hue = 'universe', ci = False, order = 2);

image.png